# Setting Workspace

In [ ]:
!pip install ipython-autotime
!pip install transformers

In [ ]:
%load_ext autotime

time: 73.4 µs (started: 2021-07-24 06:09:41 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 4.53 ms (started: 2021-07-24 06:09:41 +00:00)


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/HLT/')
!pwd

/content
/content/drive/My Drive/HLT
time: 240 ms (started: 2021-07-24 06:09:41 +00:00)


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, pipeline, TFBertModel, TFAlbertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from tensorflow.keras.layers import InputLayer, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.layers import LSTM, Dense,Flatten,Conv2D,Conv1D,GlobalMaxPooling1D,GlobalMaxPool1D
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

time: 9.89 s (started: 2021-07-24 06:09:42 +00:00)


In [ ]:
pd.set_option('display.max_colwidth', -1) #show all text in col
pd.set_option('display.max_rows', None) #show all rows

time: 1.29 ms (started: 2021-07-24 06:09:51 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


# Create albert Embeddings

In [ ]:
train_df = pd.read_csv("datasets/train.csv") 
train_df['headline']=train_df['headline'].apply(str)
train_df.head()

,headline,is_sarcastic
0,bill de blasio thinks hes proved haters wrong comes prek,0
1,baby doesnt realize white supremacist yet,1
2,spring sprung arctic way early,0
3,mayor daleys son appointed head illinois nepotist party,1
4,poll grandsons talented,1


time: 63.7 ms (started: 2021-07-24 06:09:51 +00:00)


In [ ]:
test_df = pd.read_csv("datasets/test.csv") 
test_df['headline']=test_df['headline'].apply(str)
test_df.head()

,headline,is_sarcastic
0,jessie james decker shares inspiring message postbaby bodies,0
1,frustrated russian officials struggling get policies dysfunctional trump administration,1
2,second saturday staten island art walk,0
3,chris christie emits loud sob paul ryan asks crowd whether worse years ago,1
4,chicago police department monitor interactions public using new bullet cams,1


time: 26.5 ms (started: 2021-07-24 06:09:52 +00:00)


In [ ]:
print("Train size:{}".format(train_df.shape))
print("Test size:{}".format(test_df.shape))

Train size:(22702, 2)
Test size:(4007, 2)
time: 4.44 ms (started: 2021-07-24 06:09:52 +00:00)


In [ ]:
train_y = train_df['is_sarcastic']
test_y = test_df['is_sarcastic']

train_x = train_df['headline']
test_x = test_df['headline']
train_x.shape,train_y.shape,test_x.shape,test_y.shape

((22702,), (22702,), (4007,), (4007,))

time: 7.21 ms (started: 2021-07-24 06:09:52 +00:00)


In [ ]:
# albert Embeddings

def transformer_embedding(name,inp,model_name):

    model = model_name.from_pretrained(name)
    tokenizer = AutoTokenizer.from_pretrained(name)
    pipe = pipeline('feature-extraction', model=model, 
                tokenizer=tokenizer)
    features = pipe(inp)
    features = np.squeeze(features)
    return features

time: 5.49 ms (started: 2021-07-24 06:09:52 +00:00)


In [ ]:
'''
Embedding trainset

'''
train_x_list = train_x.tolist()
train_y_list = train_y.to_list()

time: 9.71 ms (started: 2021-07-24 06:09:52 +00:00)


In [ ]:
len(train_x_list)

22702

time: 8.17 ms (started: 2021-07-24 06:09:52 +00:00)


In [ ]:
len(train_y_list)

22702

time: 4.11 ms (started: 2021-07-24 06:15:59 +00:00)


In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('albert-base-v1',train_x_list[20000:],TFAlbertModel)

Some layers from the model checkpoint at albert-base-v1 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


time: 2min 21s (started: 2021-07-24 06:15:59 +00:00)


In [ ]:
embedding_features.shape

(2702, 25, 768)

time: 6.12 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x = embedding_features[:,0,:]

time: 1.3 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x.shape

(2702, 768)

time: 4.49 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x_df = pd.DataFrame(train_albert_x)

time: 1.85 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,1.076906,0.756411,0.824502,-0.487321,0.192259,-1.182580,-0.027377,-0.400020,0.467507,0.154452,0.280550,0.372665,0.237408,0.085724,0.184971,0.133766,-0.120522,-0.732618,0.270031,0.691916,0.200623,-1.022740,0.627450,0.370138,0.742203,-1.290579,0.191444,-0.554683,-0.014610,-0.629367,-0.274807,0.436525,-0.612530,0.240716,0.339932,-0.244682,-0.586710,0.152632,0.175696,0.800564,...,0.230438,-0.137846,0.349949,0.493047,-0.283892,0.021539,0.201272,0.077770,0.290779,-0.562171,0.591492,-0.332069,-0.073569,0.197758,0.138944,-0.350052,-0.439504,-0.237852,-0.634978,-0.115303,1.219190,-0.681920,-0.607563,-0.060252,0.535440,0.409474,0.062403,0.019397,-0.373570,0.081512,-0.183766,-0.732409,-0.418798,-0.349762,1.101906,-0.282458,-0.754715,0.607596,0.590549,-0.172364
1,0.457440,0.481318,1.110510,-0.474929,-0.027705,-1.158721,-0.387082,-0.238341,0.477229,-0.014928,-0.326611,0.394764,0.249599,-0.372087,0.217692,0.740801,-0.724199,-0.439731,0.255556,1.100072,-0.140558,-0.453747,0.435989,0.214387,0.576063,-1.082650,-0.063356,0.007101,0.169863,-0.365648,0.056820,0.031582,-0.430206,0.054833,0.157029,0.342090,-0.271743,-0.294967,0.407203,0.210839,...,-0.088068,0.367558,0.367891,0.005621,-0.170045,0.201113,0.333392,0.127790,0.186264,-0.689813,-0.037596,-0.801207,0.229505,0.555374,0.294522,-0.395853,-0.644816,-0.699328,-0.054910,-0.026378,1.209829,0.548219,-0.268992,-0.031538,0.562368,-0.266632,0.661348,-0.599855,-0.018908,0.371746,-0.274462,-0.740990,-0.003166,-0.144759,0.392396,-0.401655,-1.489294,0.095153,0.350712,-0.071876
2,0.946261,0.667814,0.958797,-0.132376,0.071106,-1.027922,-0.802441,-0.245328,0.018253,-0.634006,0.913625,1.335715,-0.069673,-0.767883,-0.139091,0.203847,-0.715140,-0.927734,0.086398,0.698703,0.049547,-0.754853,1.214094,0.346602,1.046507,-2.095230,0.545144,-0.247136,0.191204,-0.176231,0.704400,1.140864,-0.477065,0.053197,-0.837995,0.617935,-0.173512,0.635927,0.108517,0.937377,...,0.385383,-0.083194,0.648572,0.450360,0.099143,1.089743,-0.357783,-0.363365,0.058509,-0.838170,0.091366,-0.761719,-0.270014,0.402420,-0.579988,-0.196718,-0.308138,-0.354262,-0.182480,-0.497102,0.513841,-0.028243,-1.156820,-0.214164,-0.211881,-0.260970,0.557705,-0.618552,-0.379329,0.546979,-0.138749,-0.717120,-0.506049,-0.158799,0.757377,-0.177777,-1.598933,0.320204,0.613596,-0.297901
3,0.674265,0.450681,0.559820,-0.256407,0.044819,-0.923412,-0.414657,0.020263,0.233769,-0.262244,-0.172156,1.009559,0.334782,-0.130197,0.048983,0.365607,-0.514139,-0.320804,0.624639,0.588353,-0.825069,-0.180026,0.101721,0.810054,1.048851,-0.857409,0.268823,0.035884,0.371577,0.292886,-0.584088,-0.033649,-0.285320,0.073066,-0.061667,0.671305,-0.946733,-0.436501,0.053883,0.231619,...,0.157606,0.182255,-0.066220,0.307295,0.163213,0.191974,-0.101786,-0.308131,0.155539,-1.063558,0.002283,-1.032958,0.533693,0.520235,0.594975,-0.632169,-0.087708,-0.752059,-0.711237,-0.332064,1.605659,-0.146599,-0.402864,0.125647,0.503436,0.098397,-0.053337,-0.276220,-0.571578,0.396679,0.079949,-0.888812,-0.476993,-0.565580,1.581243,-0.367715,-1.210728,0.201958,0.875665,0.201105
4,0.875752,0.202045,1.343674,-1.086554,0.341323,-1.396194,0.043151,-0.190039,0.414155,0.018300,0.147582,0.575963,0.449328,0.023620,-0.012159,0.108595,-0.438111,-0.635418,0.011677,0.894771,0.463112,-0.322574,0.411166,0.454433,0.554255,-1.287521,-0.134741,0.035630,0.365292,-0.322583,-0.082907,-0.489170,-0.362509,0.388607,0.443151,0.705027,-0.423465,-0.453459,0.381752,0.310488,...,0.174587,0.552284,0.255239,0.217946,-0.020635,0.311533,0.321507,-0.251960,0.010911,-0.955666,-0.054214,-0.391423,0.001534,0.336667,0.091056,-0.474717,-0.474148,-0.696548,0.279041,-0.407095,1.203746,0.213734,-0.687684,0.167537,-0.311049,-0.377292,0.582826,-0.539864,0.209331,0.270332,-

time: 123 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x_df['is_sarcastic'] = train_y_list[20000:]

time: 5.01 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,1.076906,0.756411,0.824502,-0.487321,0.192259,-1.182580,-0.027377,-0.400020,0.467507,0.154452,0.280550,0.372665,0.237408,0.085724,0.184971,0.133766,-0.120522,-0.732618,0.270031,0.691916,0.200623,-1.022740,0.627450,0.370138,0.742203,-1.290579,0.191444,-0.554683,-0.014610,-0.629367,-0.274807,0.436525,-0.612530,0.240716,0.339932,-0.244682,-0.586710,0.152632,0.175696,0.800564,...,-0.137846,0.349949,0.493047,-0.283892,0.021539,0.201272,0.077770,0.290779,-0.562171,0.591492,-0.332069,-0.073569,0.197758,0.138944,-0.350052,-0.439504,-0.237852,-0.634978,-0.115303,1.219190,-0.681920,-0.607563,-0.060252,0.535440,0.409474,0.062403,0.019397,-0.373570,0.081512,-0.183766,-0.732409,-0.418798,-0.349762,1.101906,-0.282458,-0.754715,0.607596,0.590549,-0.172364,1
1,0.457440,0.481318,1.110510,-0.474929,-0.027705,-1.158721,-0.387082,-0.238341,0.477229,-0.014928,-0.326611,0.394764,0.249599,-0.372087,0.217692,0.740801,-0.724199,-0.439731,0.255556,1.100072,-0.140558,-0.453747,0.435989,0.214387,0.576063,-1.082650,-0.063356,0.007101,0.169863,-0.365648,0.056820,0.031582,-0.430206,0.054833,0.157029,0.342090,-0.271743,-0.294967,0.407203,0.210839,...,0.367558,0.367891,0.005621,-0.170045,0.201113,0.333392,0.127790,0.186264,-0.689813,-0.037596,-0.801207,0.229505,0.555374,0.294522,-0.395853,-0.644816,-0.699328,-0.054910,-0.026378,1.209829,0.548219,-0.268992,-0.031538,0.562368,-0.266632,0.661348,-0.599855,-0.018908,0.371746,-0.274462,-0.740990,-0.003166,-0.144759,0.392396,-0.401655,-1.489294,0.095153,0.350712,-0.071876,1
2,0.946261,0.667814,0.958797,-0.132376,0.071106,-1.027922,-0.802441,-0.245328,0.018253,-0.634006,0.913625,1.335715,-0.069673,-0.767883,-0.139091,0.203847,-0.715140,-0.927734,0.086398,0.698703,0.049547,-0.754853,1.214094,0.346602,1.046507,-2.095230,0.545144,-0.247136,0.191204,-0.176231,0.704400,1.140864,-0.477065,0.053197,-0.837995,0.617935,-0.173512,0.635927,0.108517,0.937377,...,-0.083194,0.648572,0.450360,0.099143,1.089743,-0.357783,-0.363365,0.058509,-0.838170,0.091366,-0.761719,-0.270014,0.402420,-0.579988,-0.196718,-0.308138,-0.354262,-0.182480,-0.497102,0.513841,-0.028243,-1.156820,-0.214164,-0.211881,-0.260970,0.557705,-0.618552,-0.379329,0.546979,-0.138749,-0.717120,-0.506049,-0.158799,0.757377,-0.177777,-1.598933,0.320204,0.613596,-0.297901,0
3,0.674265,0.450681,0.559820,-0.256407,0.044819,-0.923412,-0.414657,0.020263,0.233769,-0.262244,-0.172156,1.009559,0.334782,-0.130197,0.048983,0.365607,-0.514139,-0.320804,0.624639,0.588353,-0.825069,-0.180026,0.101721,0.810054,1.048851,-0.857409,0.268823,0.035884,0.371577,0.292886,-0.584088,-0.033649,-0.285320,0.073066,-0.061667,0.671305,-0.946733,-0.436501,0.053883,0.231619,...,0.182255,-0.066220,0.307295,0.163213,0.191974,-0.101786,-0.308131,0.155539,-1.063558,0.002283,-1.032958,0.533693,0.520235,0.594975,-0.632169,-0.087708,-0.752059,-0.711237,-0.332064,1.605659,-0.146599,-0.402864,0.125647,0.503436,0.098397,-0.053337,-0.276220,-0.571578,0.396679,0.079949,-0.888812,-0.476993,-0.565580,1.581243,-0.367715,-1.210728,0.201958,0.875665,0.201105,1
4,0.875752,0.202045,1.343674,-1.086554,0.341323,-1.396194,0.043151,-0.190039,0.414155,0.018300,0.147582,0.575963,0.449328,0.023620,-0.012159,0.108595,-0.438111,-0.635418,0.011677,0.894771,0.463112,-0.322574,0.411166,0.454433,0.554255,-1.287521,-0.134741,0.035630,0.365292,-0.322583,-0.082907,-0.489170,-0.362509,0.388607,0.443151,0.705027,-0.423465,-0.453459,0.381752,0.310488,...,0.552284,0.255239,0.217946,-0.020635,0.311533,0.321507,-0.251960,0.010911,-0.955666,-0.054214,-0.391423,0.001534,0.336667,0.091056,-0.474717,-0.474148,-0.696548,0.279041,-0.407095,1.203746,0.213734,-0.687684,0.167537,-0.311049,-0.377292,0.582826,-0.539864,0.209331,0.270332,-0.003504,-0.842744,-0.586454,

time: 137 ms (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
train_albert_x_df.to_csv('datasets/train_albert_20000_end.csv', encoding='utf-8', index=False)

time: 3.76 s (started: 2021-07-24 06:18:20 +00:00)


In [ ]:
'''
Embedding testset

'''
test_x_list = test_x.tolist()
test_y_list = test_y.to_list()

time: 1.72 ms (started: 2021-07-24 06:19:20 +00:00)


In [ ]:
len(test_x_list)

4007

time: 3.69 ms (started: 2021-07-24 06:19:21 +00:00)


In [ ]:
len(test_y_list)

4007

time: 8.11 ms (started: 2021-07-24 06:19:22 +00:00)


In [ ]:
embedding_features=transformer_embedding('albert-base-v1',test_x_list,TFAlbertModel)

Some layers from the model checkpoint at albert-base-v1 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


time: 3min 8s (started: 2021-07-24 06:19:23 +00:00)


In [ ]:
embedding_features.shape

(4007, 23, 768)

time: 3.75 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x = embedding_features[:,0,:]

time: 1.04 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x.shape

(4007, 768)

time: 3.46 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x_df = pd.DataFrame(test_albert_x)

time: 1.33 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.801973,0.603053,0.888681,-0.139678,0.131991,-1.262391,-0.447765,-0.334753,0.549044,-0.152604,0.135961,0.943041,0.474989,-0.005787,0.253631,1.039074,-0.725516,-0.539980,0.249845,0.583471,-0.631321,-0.524529,0.267355,0.725527,1.185834,-1.368046,0.027722,0.179495,-0.286481,-0.254248,-0.112639,0.240952,-0.201319,-0.091318,-0.234163,0.067442,-0.377388,0.155789,0.094679,0.304039,...,-0.077244,0.220376,0.321677,0.036677,-0.393784,0.281884,0.486607,-0.030983,0.048349,-0.854880,0.442368,-1.340876,0.380599,0.545921,-0.001916,-0.150725,-0.743037,-0.041149,-0.057751,-0.238346,1.485241,0.295901,-0.464869,0.045014,0.738563,0.051394,0.469410,-0.659803,-0.275778,0.254020,-0.441997,-0.878847,-0.055795,0.116998,0.917978,-0.361452,-1.635254,-0.002267,0.528595,-0.280610
1,1.002656,0.740398,0.219594,-0.634089,0.398183,-1.160198,-0.181282,-0.567765,0.629236,0.101643,0.310259,0.624822,0.315918,-0.143989,0.486609,0.067528,-0.818685,-0.486813,0.143893,0.551649,0.507326,-0.488160,0.787739,0.237080,1.475216,-1.407060,0.088562,-0.760798,0.391428,-0.177607,0.055437,0.603482,-0.242257,-0.227215,-0.166374,0.712434,-0.696838,-0.125749,0.176086,0.859245,...,0.372624,-0.163412,0.179918,0.267678,-0.286115,0.327537,1.158476,0.368819,-0.279671,-0.911286,0.619588,-0.308182,0.643636,0.212605,-0.094853,-0.343132,-0.745989,-1.004064,-0.358189,-0.281112,1.020685,-0.808908,-0.487802,-0.456847,0.625132,-0.199332,0.903580,-0.229400,-0.467673,0.017187,0.028635,-1.353068,-0.512053,-0.080490,1.329885,-0.272943,-1.095978,0.164350,0.836719,-0.214160
2,0.691385,1.107759,0.547635,-0.386719,-0.084822,-1.329898,0.083970,-0.400945,0.533900,0.003895,0.753213,0.879001,0.014233,0.318564,0.245328,0.025375,-0.565803,-0.096019,0.307757,0.237374,-0.099411,-0.453366,0.568223,0.611821,0.869160,-1.670174,0.051801,-0.308719,-0.225652,-0.267139,-0.272736,0.601088,-0.085979,-0.423541,-0.337269,0.122151,-0.510880,0.203150,0.053040,1.105928,...,0.455611,0.170935,0.083997,0.593238,-0.195201,0.383640,0.780012,0.208654,0.081954,-0.952427,0.382288,-0.741685,-0.067410,0.954061,-0.355517,-0.368336,-0.159679,-0.214683,-0.810503,-0.220659,1.589236,0.081638,-0.796684,-0.167994,0.238645,0.223544,1.353035,0.099102,-0.509176,0.013050,-0.179994,-1.118093,0.014948,0.182424,1.090344,-0.142387,-1.448662,0.452157,1.048109,-0.476618
3,0.893270,1.039331,0.503443,-1.037247,-0.169415,-1.203363,0.211838,-0.185489,0.330598,0.148289,0.409837,0.278551,0.397831,0.710198,0.092568,0.178344,-0.398362,-0.932570,0.536596,0.388011,0.060820,-1.193773,0.308352,0.347444,0.486347,-1.387032,0.009895,-0.738412,0.501686,-0.553914,-0.833369,0.347566,-0.705110,0.334442,0.273077,0.001242,-0.188769,0.205982,0.097568,0.867733,...,0.426304,0.357649,0.376988,0.542030,0.218231,0.324686,0.099912,-0.025065,0.223447,-0.706305,0.867152,-0.508196,-0.090114,-0.096617,0.008487,-0.853837,-0.439057,0.139414,-0.179363,0.000604,1.222402,-0.512432,-0.513106,0.384905,-0.097693,-0.178210,0.297714,-0.297352,-0.563437,0.134374,-0.272645,-0.962577,-0.967619,-0.205314,0.904720,-0.127609,-0.694396,1.058726,0.609258,-0.264051
4,0.817568,0.982586,0.749540,-0.407883,-0.000628,-1.303250,-0.202367,0.108302,0.691777,-0.172388,0.159763,1.022441,0.603573,0.110852,0.225941,-0.032703,-0.232266,-0.379471,0.358642,0.475928,0.275095,-0.135130,0.534904,0.481351,1.508912,-1.379259,0.371294,-0.408656,0.188687,-0.052393,-0.386552,0.476051,-0.160232,-0.338153,0.072977,0.077954,-0.661874,-0.049772,0.124252,0.832755,...,0.306780,-0.476177,0.178634,0.628538,-0.451049,0.118647,0.208277,0.305522,0.045855,-1.158519,0.823739,-0.835691,0.276952,0.551242,-0.320861,-0.253768,-0.371306,-0.297202,-0.636636,-0.473984,1.889820,-0.674832,-0.415601,-0.247569,0.469139,0.319462,0.621995,0.183534,-0.742343,0.277758,-0.2981

time: 120 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x_df['is_sarcastic'] = test_y_list

time: 4.08 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,0.801973,0.603053,0.888681,-0.139678,0.131991,-1.262391,-0.447765,-0.334753,0.549044,-0.152604,0.135961,0.943041,0.474989,-0.005787,0.253631,1.039074,-0.725516,-0.539980,0.249845,0.583471,-0.631321,-0.524529,0.267355,0.725527,1.185834,-1.368046,0.027722,0.179495,-0.286481,-0.254248,-0.112639,0.240952,-0.201319,-0.091318,-0.234163,0.067442,-0.377388,0.155789,0.094679,0.304039,...,0.220376,0.321677,0.036677,-0.393784,0.281884,0.486607,-0.030983,0.048349,-0.854880,0.442368,-1.340876,0.380599,0.545921,-0.001916,-0.150725,-0.743037,-0.041149,-0.057751,-0.238346,1.485241,0.295901,-0.464869,0.045014,0.738563,0.051394,0.469410,-0.659803,-0.275778,0.254020,-0.441997,-0.878847,-0.055795,0.116998,0.917978,-0.361452,-1.635254,-0.002267,0.528595,-0.280610,0
1,1.002656,0.740398,0.219594,-0.634089,0.398183,-1.160198,-0.181282,-0.567765,0.629236,0.101643,0.310259,0.624822,0.315918,-0.143989,0.486609,0.067528,-0.818685,-0.486813,0.143893,0.551649,0.507326,-0.488160,0.787739,0.237080,1.475216,-1.407060,0.088562,-0.760798,0.391428,-0.177607,0.055437,0.603482,-0.242257,-0.227215,-0.166374,0.712434,-0.696838,-0.125749,0.176086,0.859245,...,-0.163412,0.179918,0.267678,-0.286115,0.327537,1.158476,0.368819,-0.279671,-0.911286,0.619588,-0.308182,0.643636,0.212605,-0.094853,-0.343132,-0.745989,-1.004064,-0.358189,-0.281112,1.020685,-0.808908,-0.487802,-0.456847,0.625132,-0.199332,0.903580,-0.229400,-0.467673,0.017187,0.028635,-1.353068,-0.512053,-0.080490,1.329885,-0.272943,-1.095978,0.164350,0.836719,-0.214160,1
2,0.691385,1.107759,0.547635,-0.386719,-0.084822,-1.329898,0.083970,-0.400945,0.533900,0.003895,0.753213,0.879001,0.014233,0.318564,0.245328,0.025375,-0.565803,-0.096019,0.307757,0.237374,-0.099411,-0.453366,0.568223,0.611821,0.869160,-1.670174,0.051801,-0.308719,-0.225652,-0.267139,-0.272736,0.601088,-0.085979,-0.423541,-0.337269,0.122151,-0.510880,0.203150,0.053040,1.105928,...,0.170935,0.083997,0.593238,-0.195201,0.383640,0.780012,0.208654,0.081954,-0.952427,0.382288,-0.741685,-0.067410,0.954061,-0.355517,-0.368336,-0.159679,-0.214683,-0.810503,-0.220659,1.589236,0.081638,-0.796684,-0.167994,0.238645,0.223544,1.353035,0.099102,-0.509176,0.013050,-0.179994,-1.118093,0.014948,0.182424,1.090344,-0.142387,-1.448662,0.452157,1.048109,-0.476618,0
3,0.893270,1.039331,0.503443,-1.037247,-0.169415,-1.203363,0.211838,-0.185489,0.330598,0.148289,0.409837,0.278551,0.397831,0.710198,0.092568,0.178344,-0.398362,-0.932570,0.536596,0.388011,0.060820,-1.193773,0.308352,0.347444,0.486347,-1.387032,0.009895,-0.738412,0.501686,-0.553914,-0.833369,0.347566,-0.705110,0.334442,0.273077,0.001242,-0.188769,0.205982,0.097568,0.867733,...,0.357649,0.376988,0.542030,0.218231,0.324686,0.099912,-0.025065,0.223447,-0.706305,0.867152,-0.508196,-0.090114,-0.096617,0.008487,-0.853837,-0.439057,0.139414,-0.179363,0.000604,1.222402,-0.512432,-0.513106,0.384905,-0.097693,-0.178210,0.297714,-0.297352,-0.563437,0.134374,-0.272645,-0.962577,-0.967619,-0.205314,0.904720,-0.127609,-0.694396,1.058726,0.609258,-0.264051,1
4,0.817568,0.982586,0.749540,-0.407883,-0.000628,-1.303250,-0.202367,0.108302,0.691777,-0.172388,0.159763,1.022441,0.603573,0.110852,0.225941,-0.032703,-0.232266,-0.379471,0.358642,0.475928,0.275095,-0.135130,0.534904,0.481351,1.508912,-1.379259,0.371294,-0.408656,0.188687,-0.052393,-0.386552,0.476051,-0.160232,-0.338153,0.072977,0.077954,-0.661874,-0.049772,0.124252,0.832755,...,-0.476177,0.178634,0.628538,-0.451049,0.118647,0.208277,0.305522,0.045855,-1.158519,0.823739,-0.835691,0.276952,0.551242,-0.320861,-0.253768,-0.371306,-0.297202,-0.636636,-0.473984,1.889820,-0.674832,-0.415601,-0.247569,0.469139,0.319462,0.621995,0.183534,-0.742343,0.277758,-0.298104,-1.314303,-0.450149,-0.204

time: 142 ms (started: 2021-07-24 06:22:31 +00:00)


In [ ]:
test_albert_x_df.to_csv('datasets/test_albert.csv', encoding='utf-8', index=False)

time: 5.76 s (started: 2021-07-24 06:22:32 +00:00)


# Merge Trainset


In [ ]:
train_albert_1 = pd.read_csv("datasets/train_albert_0_4000.csv") 
train_albert_2 = pd.read_csv("datasets/train_albert_4000_8000.csv")
train_albert_3 = pd.read_csv("datasets/train_albert_8000_12000.csv") 
train_albert_4 = pd.read_csv("datasets/train_albert_12000_16000.csv")
train_albert_5 = pd.read_csv("datasets/train_albert_16000_20000.csv") 
train_albert_6 = pd.read_csv("datasets/train_albert_20000_end.csv") 

time: 9.3 s (started: 2021-07-24 06:23:40 +00:00)


In [ ]:
train_albert_df = train_albert_1.append([train_albert_2, train_albert_3, train_albert_4, train_albert_5, train_albert_6])

time: 49 ms (started: 2021-07-24 06:23:49 +00:00)


In [ ]:
train_albert_df.shape

(22702, 769)

time: 4.09 ms (started: 2021-07-24 06:23:49 +00:00)


In [ ]:
train_albert_df.to_csv('datasets/train_albert.csv', encoding='utf-8', index=False)

time: 33.6 s (started: 2021-07-24 06:23:49 +00:00)


# Statistical Models

In [ ]:
train_albert_df = pd.read_csv("datasets/train_albert.csv") 
train_albert_df.shape 

(22702, 769)

time: 4.76 s (started: 2021-07-24 06:24:23 +00:00)


In [ ]:
train_albert_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,1.023280,0.895187,0.989915,-0.169690,0.441390,-1.390489,-0.281808,-0.525141,0.387006,-0.017008,-0.187903,0.903642,0.264225,-0.267946,0.541225,0.273503,-0.430852,-0.503543,0.413821,0.659606,0.485305,-0.847754,0.443774,0.435464,1.034831,-1.388442,0.002525,-0.529567,-0.243076,-0.350295,-0.421348,0.578152,-0.796267,0.148488,-0.229350,-0.008867,-0.819185,0.123969,0.191214,0.449084,...,-0.021556,0.447038,0.086751,-0.053924,0.651017,0.394048,-0.132330,0.129641,-0.752214,0.480866,-1.047732,-0.301873,0.343337,0.265263,-0.513996,-0.605785,-0.305817,-0.079084,-0.434675,1.138793,-0.325433,-0.885293,0.167673,0.354164,0.002945,0.574502,-0.471479,0.253475,0.121056,0.364238,-0.656706,-0.233825,-0.245490,1.031559,-0.271107,-1.271443,0.619048,0.733553,-0.344365,0
1,0.449931,0.107994,1.609914,-0.733145,0.333371,-1.484948,-0.556294,-0.390717,0.267236,-0.144634,0.455046,-0.008190,0.311757,-0.237607,-0.237178,0.070377,-0.364445,-0.292035,-0.277334,0.905621,0.037692,-0.410012,0.535655,-0.005520,0.501699,-1.614345,0.344905,-0.404084,0.077690,-1.007267,0.039021,0.137657,-0.464374,0.179157,0.264206,0.167932,-0.190391,-0.502667,0.417256,0.411249,...,-0.068391,0.314582,0.144037,0.006905,0.557222,0.047203,0.021880,0.225886,-0.805515,0.475126,-0.115361,-0.361877,0.021025,-0.063544,-0.472860,-0.324740,-0.991788,-0.286805,-0.399642,1.608840,0.483521,-0.577067,0.137920,-0.931260,0.052697,-0.093326,-0.647421,0.018980,0.914664,0.248234,-0.690462,-0.110769,0.058490,0.604097,-0.312212,-0.702721,0.279630,0.237054,-0.048657,1
2,0.601503,0.392882,1.303799,-0.880750,0.148182,-1.004196,-0.115687,-0.386831,0.367855,-0.400426,1.051681,-0.074451,0.269480,0.030576,-0.221407,-0.159435,-0.372431,-0.395086,-0.184329,0.849530,0.295308,-0.482874,0.493594,0.707158,0.269746,-1.170135,-0.003407,-0.140821,-0.006439,-0.991943,0.017885,-0.466678,-0.221326,-0.030009,0.250098,0.981158,0.189782,-0.698839,0.195924,0.426928,...,0.053805,0.216817,0.261911,0.253502,0.483365,0.487380,0.232373,-0.299052,-0.947448,0.026449,-0.095607,0.120775,0.133538,0.171679,-0.545684,-0.117664,-1.489263,-0.061904,-0.109398,1.716394,0.429177,-0.939798,0.147342,-0.571610,-0.370212,0.057980,-0.607656,-0.123695,0.833845,0.200891,-0.571208,0.079142,-0.118473,1.043789,-0.245240,-0.693814,0.285140,0.466300,0.042190,0
3,0.944271,1.285116,0.326767,-0.408492,-0.100821,-1.291777,-0.200796,-0.540462,0.437598,-0.121134,0.271774,0.602744,-0.050901,0.063214,0.213802,0.079949,-0.503838,-0.125577,0.124323,0.080048,0.336236,-0.719454,0.545017,0.463074,0.848937,-1.191145,-0.215493,-0.759938,0.026334,-0.232196,-0.002080,0.390002,0.083463,-0.117988,0.034452,0.354538,-0.233253,-0.164120,0.099990,0.664759,...,-0.505520,0.130106,0.733794,-0.309592,0.391989,1.146057,0.432787,0.013609,-1.257140,0.671418,-0.501974,0.198930,0.305021,-0.084897,-0.474736,-0.276442,-0.410528,-0.874765,-0.231103,1.178519,-0.426512,-0.337678,-0.460124,1.025718,0.119895,0.931806,-0.195901,-0.464796,0.047539,0.097426,-0.983090,-0.062953,-0.137388,1.525907,-0.174638,-0.542520,0.459630,0.868670,-0.346602,1
4,0.318234,0.158980,0.377982,0.136996,0.137129,-0.911632,-0.605344,-0.421252,-0.005661,-0.183056,-0.080016,0.880818,0.475075,-0.549752,0.244365,0.518229,-0.712092,-0.380384,0.167159,0.297190,-0.758429,-0.222386,0.207714,0.506053,0.443125,-1.468624,0.035308,-0.591255,-0.127466,0.288922,-0.074708,0.160430,-0.123131,0.165306,-0.688239,0.040621,-1.051906,0.056861,-0.146007,0.361549,...,0.349046,0.166736,0.035535,0.086408,0.411699,0.181582,-0.313081,0.068449,-1.157538,-0.039016,-0.819712,0.173694,0.421779,1.095462,-0.312308,-0.284654,-0.391196,-0.399884,-0.206713,1.861338,0.090706,-0.495511,0.051510,0.272552,0.115048,0.780309,-0.695842,-0.374129,0.511418,0.163815,-0.431575,-0.149110

time: 113 ms (started: 2021-07-24 06:24:28 +00:00)


In [ ]:
test_albert_df = pd.read_csv("datasets/test_albert.csv") 
test_albert_df.shape 

(4007, 769)

time: 947 ms (started: 2021-07-24 06:24:28 +00:00)


In [ ]:
train_y = train_albert_df.pop('is_sarcastic')
test_y = test_albert_df.pop('is_sarcastic')

train_x = train_albert_df
test_x = test_albert_df

train_x.shape,train_y.shape,test_x.shape,test_y.shape

((22702, 768), (22702,), (4007, 768), (4007,))

time: 9.9 ms (started: 2021-07-24 06:24:29 +00:00)


## Logistic Regression

In [ ]:
# Applying Logistic Regression on split tfidf baseline
model=LogisticRegression(max_iter=10000)
model.fit(train_x,train_y)
pred=model.predict(test_x)
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1753  495]
 [ 576 1183]]
Accuracy Score for LR with C=1.0  =0.7327177439480909
time: 23.2 s (started: 2021-07-24 06:24:29 +00:00)


## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25, weights = 'distance')
knn.fit(train_x,train_y)
pred=knn.predict(test_x)

time: 2min 47s (started: 2021-07-24 06:24:52 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1761  487]
 [ 740 1019]]
Accuracy Score for LR with C=1.0  =0.6937858747192414
time: 8.52 ms (started: 2021-07-24 06:27:40 +00:00)


## Decision Tree

In [ ]:
dt = DecisionTreeClassifier(criterion ='entropy', max_depth= None, min_samples_leaf=1, min_samples_split=20,splitter="best")
dt.fit(train_x,train_y)
pred=dt.predict(test_x)

time: 52 s (started: 2021-07-24 06:27:40 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1414  834]
 [ 815  944]]
Accuracy Score for LR with C=1.0  =0.5884701771899177
time: 13.6 ms (started: 2021-07-24 06:28:32 +00:00)


## Random Forest

In [ ]:
rf = RandomForestClassifier(criterion='entropy',n_estimators = 100, max_depth = None, bootstrap = False)
rf.fit(train_x,train_y)
pred=rf.predict(test_x)

time: 3min 21s (started: 2021-07-24 06:28:32 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1840  408]
 [ 848  911]]
Accuracy Score for LR with C=1.0  =0.686548540054904
time: 11.1 ms (started: 2021-07-24 06:31:54 +00:00)


## Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(train_x,train_y)
pred=gnb.predict(test_x)

time: 340 ms (started: 2021-07-24 06:31:54 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1381  867]
 [ 814  945]]
Accuracy Score for LR with C=1.0  =0.5804841527327177
time: 11.5 ms (started: 2021-07-24 06:31:54 +00:00)


## SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel ='rbf', gamma = 0.1, C = 10, probability=True) 
svc.fit(train_x,train_y)
pred=svc.predict(test_x)

time: 3h 15min 33s (started: 2021-07-24 06:31:54 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[2042  206]
 [1143  616]]
Accuracy Score for LR with C=1.0  =0.6633391564761667
time: 21.4 ms (started: 2021-07-24 09:47:28 +00:00)
